In [25]:
import torch
from executorch.backends.apple.coreml.partition import CoreMLPartitioner

In [ ]:
exported_module = torch.export.load("./bin/nemo_decoder.pt2")

In [27]:
import coremltools
from executorch.backends.apple.coreml.compiler import CoreMLBackend
import executorch.exir as exir

_EDGE_COMPILE_CONFIG = exir.EdgeCompileConfig(
    _check_ir_validity=False,
    _skip_dim_order=True,
)

compile_specs = CoreMLBackend.generate_compile_specs(
    minimum_deployment_target=coremltools.target.iOS16,
    compute_precision=coremltools.precision.FLOAT16,
    compute_unit=coremltools.ComputeUnit.CPU_AND_GPU,
)

et_program = exir.to_edge(exported_module, compile_config=_EDGE_COMPILE_CONFIG)

/Users/jakubchmura/Desktop/SWM_AI/executorch/.venv/lib/python3.10/site-packages/torch/export/_unlift.py:81: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/Users/jakubchmura/Desktop/SWM_AI/executorch/.venv/lib/python3.10/site-packages/torch/fx/graph.py:1772: UserWarning: Node lifted_tensor_0 target lifted_tensor_0 lifted_tensor_0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(


In [28]:
partitioner = CoreMLPartitioner(
    skip_ops_for_coreml_delegation=None, compile_specs=compile_specs,
)
delegated_program_manager = et_program.to_backend(partitioner)
exec_program = delegated_program_manager.to_executorch(
    config=exir.ExecutorchBackendConfig(extract_delegate_segments=True)
)

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 236.14 passes/s]
/Users/jakubchmura/Desktop/SWM_AI/executorch/.venv/lib/python3.10/site-packages/torch/export/_unlift.py:81: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/Users/jakubchmura/Desktop/SWM_AI/executorch/.venv/lib/python3.10/site-packages/torch/fx/graph.py:1772: UserWarning: Node lifted_tensor_0 target lifted_tensor_0 lifted_tensor_0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/Users/jakubchmura/Desktop/SWM_AI/executorch/.venv/lib/python3.10/site-packages/torch/export/_unlift.py:81: UserWarning: Attempted to insert a get_attr Node with n

In [ ]:
import os
from pathlib import Path

filename = Path(os.getcwd(), "bin", "nemo_decoder_coreml.pte")
with open(filename, "wb") as file:
    exec_program.write_to_file(file)

In [30]:
from pathlib import Path

import torch
from executorch.runtime import Verification, Runtime, Program, Method

et_runtime: Runtime = Runtime.get()
program: Program = et_runtime.load_program(
    Path(filename),
    verification=Verification.Minimal,
)
forward: Method = program.load_method("forward")

encoder_hidden_states = torch.rand(1, 35, 1024)
decoder_input_ids = torch.randint(0, 500, (1, 9), dtype=torch.int64)

forward.execute((decoder_input_ids, encoder_hidden_states, ))

[tensor([[ 55, 127, 473,  90, 375, 262, 433, 235,  69,   3]])]

In [31]:
%%timeit
forward.execute((decoder_input_ids, encoder_hidden_states))

1.9 ms ± 49.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
